In [1]:
!pip install facenet-pytorch


In [2]:
import torch
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision import transforms
from google.colab import files


In [3]:
# Load the pretrained model
model = InceptionResnetV1(pretrained='vggface2').eval()


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [18]:
uploaded = files.upload()  # Select your image file
img_path = list(uploaded.keys())[0]


Saving F100666.jpg to F100666 (1).jpg


In [19]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

img = Image.open(img_path).convert('RGB')
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension


In [20]:
with torch.no_grad():
    embedding = model(img_tensor)




Embedding shape: torch.Size([1, 512])
Embedding vector: tensor([[-4.5447e-02,  3.0283e-02,  2.2116e-02, -4.8703e-02,  4.4408e-02,
         -4.1419e-02,  2.8092e-02, -1.7078e-02,  4.8241e-02,  3.7053e-02,
          3.1917e-02,  3.3305e-03, -2.8682e-02, -6.3701e-02,  8.2612e-03,
          6.5953e-03,  1.3197e-02, -6.5251e-02, -5.6304e-02, -2.1831e-02,
         -1.0040e-02,  3.2393e-02,  3.5782e-02,  3.4780e-02, -5.7846e-02,
          4.6968e-02,  2.9808e-02, -6.9357e-02, -1.5334e-02, -2.1324e-02,
         -4.7402e-02,  8.5069e-02,  1.0670e-02,  2.7910e-02, -4.7694e-03,
         -3.7018e-04,  5.3910e-02, -5.4024e-02, -2.5230e-02,  1.4724e-03,
          8.2834e-02, -3.3628e-02, -3.6524e-02,  6.8842e-02, -3.0748e-02,
         -2.4508e-02,  2.5608e-02,  7.5551e-02, -3.0495e-02,  9.7641e-03,
          3.4775e-03,  9.5186e-02,  7.8868e-03, -5.6067e-02, -1.3755e-04,
          2.7887e-02, -4.4612e-02, -4.1188e-04, -3.6495e-02,  1.6856e-02,
         -4.1302e-03,  1.9159e-02, -4.9776e-02, -8.6979e

In [7]:
import pandas as pd
import numpy as np

In [9]:


df = pd.read_csv('face_embeddings.csv', on_bad_lines='skip')  # skips bad rows


In [10]:
import torch
import ast

# Convert embedding strings to PyTorch tensors
df['embedding_tensor'] = df['embedding'].apply(lambda x: torch.tensor(ast.literal_eval(x)))


In [21]:
# Suppose this is your new embedding from InceptionResnetV1
new_embedding = embedding.squeeze()  # shape [512]


In [22]:
# Compute distance to each embedding
df['distance'] = df['embedding_tensor'].apply(lambda x: torch.norm(x - new_embedding))
similar_face = df.loc[df['distance'].idxmin()]
print("Most similar face (Euclidean):", similar_face['face_id'])


Most similar face (Euclidean): F100666.jpg
